In [ ]:
import os
import numpy as np
import pandas as pd
import random
import seaborn as sns

import datetime as datetime
import matplotlib.dates as dates
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from contextlib import contextmanager
from time import time
from tqdm import tqdm
import lightgbm as lgbm

from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

# data : tokyo_covid19_patients.csv

In [ ]:
data00 = pd.read_csv("../input/covid19-tokyo/tokyo_covid19_patients - 0814.csv")
data00[-5:]

In [ ]:
data00['pcr_positives']=1

In [ ]:
data0=data00.groupby('date',as_index=False).sum()
data0[-10:]

In [ ]:
data0['ratio to 7 days before']=1

In [ ]:
n=len(data0)
for i in range(n):    
    cpi=data0['pcr_positives'][i]
    data0.loc[i+7,'pcr_positives 7 days before']=cpi
data0[100:105]

In [ ]:
data0['ratio to 7 days before']=data0['pcr_positives']*100/data0['pcr_positives 7 days before']
data0['mean ratio to 7 days before']=data0['ratio to 7 days before'].rolling(window=7).mean()
data1=data0[407:-7]
data1

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data1['date'],y=data1['mean ratio to 7 days before'],name='mean ratio to 7 days before'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Mean Ratio of icreasing to 7 days before in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Ratio",secondary_y=False)
fig.show()

### The peak of ratio of increasing was 221 on 2021-08-01.  

In [ ]:
data2=data1[-7:]
data2

## the latest mean ratio of weekly increase

In [ ]:
factor=round(data2.loc[data2.index.tolist()[-1],'mean ratio to 7 days before'])/100
print(factor)

In [ ]:
N=list(range(40))
NUMB=pd.DataFrame(N)

In [ ]:
data2['date2']=pd.to_datetime(data2['date'])
data2=pd.concat([data2,NUMB],axis=0)
data2=data2.drop([0,'date'],axis=1).reset_index(drop=True)
data2[0:10]

In [ ]:
data2['pcr_positives_ft']=data2['pcr_positives']
#data2['pcr_positives180']=data2['pcr_positives']
#data2['pcr_positives170']=data2['pcr_positives']
#data2['pcr_positives150']=data2['pcr_positives']
#data2['pcr_positives130']=data2['pcr_positives']
data2['pcr_positives110']=data2['pcr_positives']

In [ ]:
for i in range(7,47):
    data2.loc[i,'date2']=data2.loc[i-1,'date2']+datetime.timedelta(days=1)
    data2.loc[i,'pcr_positives_ft']=data2.loc[i-7,'pcr_positives_ft']*factor
    #data2.loc[i,'pcr_positives180']=data2.loc[i-7,'pcr_positives180']*1.8       
    #data2.loc[i,'pcr_positives170']=data2.loc[i-7,'pcr_positives170']*1.7   
    #data2.loc[i,'pcr_positives150']=data2.loc[i-7,'pcr_positives150']*1.5    
    #data2.loc[i,'pcr_positives130']=data2.loc[i-7,'pcr_positives130']*1.3
    data2.loc[i,'pcr_positives110']=data2.loc[i-7,'pcr_positives110']*1.1  
data3=data2[['date2','pcr_positives','pcr_positives_ft','pcr_positives110']]
data3

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives_ft'],name='the latest incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives180'],name='180% incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives170'],name='170% incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives150'],name='150% incresase weekly'),secondary_y=False,)
#fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives130'],name='130% incresase weekly'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives110'],name='110% incresase weekly'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data3['date2'],y=data3['pcr_positives'],name='actual pcr_positives'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="Prediction of pcr_positives in Tokyo")
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Number",secondary_y=False)
fig.show()

### The prediciton as of 2021-08-12
* the latest ratio of increase is 116%.
* The number of positives would exceed 10,000 on 2021-09-09.

### The prediciton as of 2021-08-07
* the latest ratio of increase is 138%.
* The number of positives would exceed 10,000 on 2021-08-25, and 50,000 on 2021-08-28.

### The prediciton as of 2021-08-05
* the latest ratio of increase is 189%.
* The number of positives would exceed 10,000 on 2021-08-25.

### The prediciton as of 2021-07-31
* the latest ratio of increase is 221%.
* The number of positives would exceed 10,000 on 2021-08-10, and 50,000 on 2021-08-24. ==> Wrong!

### The prediciton as of 2021-07-30
* the latest ratio of increase is 181%.
* The number of positives would exceed 10,000 on 2021-08-11, and 50,000 on 2021-08-31. ==> Wrong!

### The prediciton as of 2021-07-29
* the latest ratio of increase is 161%. 
* The number of positives would exceed 10,000 on 2021-08-12. ==> Wrong!